In [1]:
import de.l3s.archivespark.ArchiveSpark
import de.l3s.archivespark.enrich.functions._
import de.l3s.archivespark.nativescala.implicits._
import de.l3s.archivespark.implicits._

In [2]:
implicit val sparkContext = sc

In [3]:
val warcPath = "C:\\Users\\holzmann\\L3S\\Workspace\\archives_unleashed\\sample"
val cdxPath = s"${warcPath}\\*.cdx.gz"

In [4]:
val rdd = ArchiveSpark.hdfs(cdxPath, warcPath)

In [5]:
rdd.filter(r => r.status == 200 && r.mime == "text/html").enrich(StringContent).take(1).head.toJsonString

{
  "record":{
    "redirectUrl":"-",
    "timestamp":"20111222161517",
    "digest":"LOD6KYDGJO2KR3W4ONCSAD5ZHP27RKLJ",
    "originalUrl":"http://blog.alexanderhiggins.com/2011/06/09/cnn-48-great-depression-coming-year-24481/",
    "surtUrl":"com,alexanderhiggins,blog)/2011/06/09/cnn-48-great-depression-coming-year-24481",
    "mime":"text/html",
    "meta":"-",
    "status":200
  },
  "payload":{
    "string":"<!DOCTYPE html PUBLIC \"-//W3C//DTD XHTML 1.0 Transitional//EN\" \"http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd\">\r\n\r\n<!--\r\nAlexander Higgins Blog uses HeatMap Theme 2.5.4 (http://heatmaptheme.com)\r\nby Stuart Wider (http://stuartwider.com) copyright 2010\r\n-->\r\n\r\n<!-- \r\nheader.php\r\n~~~~~~~~~~~~~~~~~~~~~~~~ ~~~ -->\r\n\r\n<html xmlns=\"...

In [6]:
val gooseEntities = rdd.filter(r => r.status == 200 && r.mime == "text/html").enrich{
    val conf = Goose.newConfig
    conf.enableImageFetching = false
    Goose(conf)
}.enrich(Entities.of(Goose, "text"))

In [7]:
println(gooseEntities.take(11).drop(10).head.toJsonString)

{
  "record":{
    "redirectUrl":"-",
    "timestamp":"20120203023156",
    "digest":"SV56XHQKSVYJQNZRA4MVYR6Y6Q55TD7D",
    "originalUrl":"http://occupywallst.org/forum/al-sharpton-or-bill-cosby-who-has-done-more-to-adv/",
    "surtUrl":"org,occupywallst)/forum/al-sharpton-or-bill-cosby-who-has-done-more-to-adv",
    "mime":"text/html",
    "meta":"-",
    "status":200
  },
  "payload":{
    "string":{
      "goose":{
        "image":"",
        "text":{
          "_":"Posted 2 months ago on Nov. 6, 2011, 11:56 a.m. EST by MikeyD (585) from Alameda, CA This content is user submitted and not an official statement\n\nI can see the How is this relevant questions already. Let me tie it in.\n\nOne pushes a philosophy of blame others for your problems, the other pushes a philosophy of personal responsibility. Which is better?\n\nHave things gotten better for black people under Sharpton, Jackson, Wright, Lee, Waters, or Obama?\n\nWould using Cosby, Powell, West, Rice, Cain, Thomas, or Payne 

In [8]:
gooseEntities.saveToEs("histdiv/archive-records")